# [T256075](https://phabricator.wikimedia.org/T256075) Post Deployment Data QA - Collapsible Sidebar Instrumentation

In this task we will be verifying if events fired in the test wikis are making it to the schema [DesktopWebUIActionsTracking](https://meta.wikimedia.org/wiki/Schema:DesktopWebUIActionsTracking) and if the Instrumentation is working correctly.  
**Part 1:**
- euwiki  
- frwiktionary
- ptwikiversity

**Part 2:**
- frwiki
- hewiki
- fawiki

In [1]:
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import numpy as np
import pandas as pd
from tabulate import tabulate
from wmfdata import charting, hive, mariadb
from wmfdata.charting import comma_fmt, pct_fmt
from wmfdata.utils import df_to_remarkup, pct_str

You are using wmfdata v1.0.1, but v1.0.4 is available.

To update, run `pip install --upgrade git+https://github.com/neilpquinn/wmfdata/wmfdata.git@release`.

To see the changes, refer to https://github.com/neilpquinn/wmfdata/blob/release/CHANGELOG.md


### Count events by Wiki

In [2]:
# Check by Wiki 

count_actions_wiki = hive.run("""
SELECT
  date_format(dt, "yyyy-MM-dd") AS date,
  event.action AS action,
  wiki AS wiki,
  COUNT(*) AS events
FROM event.desktopwebuiactionstracking
WHERE
  year = 2020 
  AND month=7 AND day > 15   
GROUP BY date_format(dt, "yyyy-MM-dd"), event.action, wiki
""")

In [3]:
count_actions_wiki.pivot_table(index=["wiki","date"],columns=["action"])

events        
action                    click    init
wiki          date                     
enwiki        2020-07-20    NaN     1.0
euwiki        2020-07-22   67.0   529.0
frwiktionary  2020-07-22  130.0  8635.0
ptwikiversity 2020-07-22    2.0    68.0
testwiki      2020-07-16  219.0   633.0
              2020-07-17  234.0   690.0
              2020-07-18   63.0   188.0
              2020-07-19  101.0   256.0
              2020-07-20  116.0   285.0
              2020-07-21  640.0   640.0
              2020-07-22  176.0   417.0

### Count events by skinVersion

In [4]:
# Check by skinVersion 

count_actions_skinVersion = hive.run("""
SELECT
  event.action AS action,
  event.skinversion as skinversion,
  COUNT(*) AS events
FROM event.desktopwebuiactionstracking
WHERE
  year = 2020 
  AND month=7 AND day > 15   
GROUP BY event.action, event.skinversion
""")

In [5]:
count_actions_skinVersion.pivot(index='skinversion', columns='action')

events      
action       click  init
skinversion             
1             1032  3423
2              716  8919

### Count of events by skinVersion on test wikis

In [7]:
# Check by skinVersion and wiki

count_actions_skinVersion_wiki = hive.run("""
SELECT
  event.action AS action,
  event.skinversion as skinversion,
  wiki AS wiki,
  COUNT(*) AS events
FROM event.desktopwebuiactionstracking
WHERE
  year = 2020 
  AND month=7 AND day > 15   
GROUP BY event.action, event.skinversion, wiki
""")

In [11]:
count_actions_skinVersion_wiki.pivot_table(index=['wiki','skinversion'],columns=['action']).fillna(0)

events        
action                      click    init
wiki          skinversion                
enwiki        1               0.0     1.0
euwiki        1               5.0    53.0
              2              62.0   476.0
frwiktionary  1              18.0   663.0
              2             112.0  7972.0
ptwikiversity 1               2.0     9.0
              2               0.0    59.0
testwiki      1            1007.0  2697.0
              2             542.0   412.0

### Clicks to the Collapsible Sidebar

In [12]:
# Collapsible sidebar events by Date

count_actions_sidebar = hive.run("""
SELECT
  date_format(dt, "yyyy-MM-dd") AS date,
  event.name as name,
  COUNT(*) AS events
FROM event.desktopwebuiactionstracking
WHERE
  year = 2020 
  AND month=7 and day>15
  AND event.name LIKE 'ui.%'
GROUP BY date_format(dt, "yyyy-MM-dd"), event.name
""")

In [13]:
count_actions_sidebar.pivot(index='date', columns='name')

,events
name,ui.sidebar
date,
2020-07-16,9
2020-07-17,23
2020-07-18,8
2020-07-19,6
2020-07-20,28
2020-07-22,165


In [16]:
# Collapsible sidebar events by Wiki and date

count_actions_sidebarwiki = hive.run("""
SELECT
  date_format(dt, "yyyy-MM-dd") AS date,
  event.name as name,
  wiki AS wiki,
  COUNT(*) AS events
FROM event.desktopwebuiactionstracking
WHERE
  year = 2020 
  AND month=7 and day>18
  AND event.name LIKE 'ui.%'
GROUP BY date_format(dt, "yyyy-MM-dd"), event.name, wiki
""")

In [17]:
count_actions_sidebarwiki.pivot_table(index=['wiki','date'], columns=['name'])

events
name                    ui.sidebar
wiki         date                 
euwiki       2020-07-22         45
frwiktionary 2020-07-22        115
testwiki     2020-07-19          6
             2020-07-20         28
             2020-07-22          5

## Donate link events
With the implementation of the Collapsible Sidebar, we are interested to understand in particular, what is the overall difference in interaction with the donate link?

**Donate link name = "n-sitesupport"**


In [22]:
# Donate link events

count_donate_events = hive.run("""
SELECT
  date_format(dt, "yyyy-MM-dd") AS date,
  event.name as name,
  COUNT(*) AS events
FROM event.desktopwebuiactionstracking
WHERE
  year = 2020 
  AND month=7 and day>15
  AND event.name LIKE 'n-site%'
GROUP BY date_format(dt, "yyyy-MM-dd"), event.name
""")

In [23]:
count_donate_events

,date,name,events
0,2020-07-16,n-sitesupport,2
1,2020-07-18,n-sitesupport,1
2,2020-07-20,n-sitesupport,22
3,2020-07-21,n-sitesupport,80


In [29]:
# Donate link events by Wiki

count_donate_wiki = hive.run("""
SELECT
  date_format(dt, "yyyy-MM-dd") AS date,
  event.name as name,
  wiki AS wiki,
  COUNT(*) AS events
FROM event.desktopwebuiactionstracking
WHERE
  year = 2020 
  AND month=7 and day>14
  AND event.name LIKE 'n-site%'
GROUP BY date_format(dt, "yyyy-MM-dd"), event.name, wiki
""")

No Clicks recorded on the Donate link on any of the 6 test wikis so far

In [30]:
count_donate_wiki.pivot_table(index=['wiki','name'], columns=['date'])

events                                            
date                   2020-07-15 2020-07-16 2020-07-18 2020-07-20 2020-07-21
wiki     name                                                                
testwiki n-sitesupport          9          2          1         22         80

In [33]:
# Donate link events on Wikis by skinVersion

count_donate_skinVersion= hive.run("""
SELECT
  event.skinversion,
  event.name as name,
  wiki AS wiki,
  COUNT(*) AS events
FROM event.desktopwebuiactionstracking
WHERE
  year = 2020 
  AND month=7 AND day>15
  AND event.name LIKE 'n-sitesupport'
GROUP BY event.skinversion, event.name, wiki
""")

In [39]:
count_donate_skinVersion.pivot_table(index=['name','wiki','skinversion'])

events
name          wiki     skinversion        
n-sitesupport testwiki 1                12
                       2                93

In [41]:
# Donate link events by logged in/out users

count_donate_skinVersion_anon= hive.run("""
SELECT
  event.skinversion,
  event.name as name,
  wiki AS wiki,
  event.isanon AS anonymous_user,
  COUNT(*) AS events
FROM event.desktopwebuiactionstracking
WHERE
  year = 2020 
  AND month=7 AND day>15
  AND event.name LIKE 'n-sitesupport'
GROUP BY event.skinversion, event.name, wiki, event.isanon 
""")

In [47]:
count_donate_skinVersion_anon.pivot_table(index=['name','wiki','skinversion'], columns=['anonymous_user']).fillna(0)

events      
anonymous_user                      False True 
name          wiki     skinversion             
n-sitesupport testwiki 1              0.0  12.0
                       2             83.0  10.0

In [48]:
# Donate link events by logged in/out users by day

count_donate_skinVersion_anon_day= hive.run("""
SELECT
  date_format(dt, "yyyy-MM-dd") AS date,
  event.skinversion,
  event.name as name,
  wiki AS wiki,
  event.isanon AS anonymous_user,
  COUNT(*) AS events
FROM event.desktopwebuiactionstracking
WHERE
  year = 2020 
  AND month=7 AND day>15
  AND event.name LIKE 'n-sitesupport'
GROUP BY date_format(dt, "yyyy-MM-dd"),event.skinversion, event.name, wiki, event.isanon 
""")

In [49]:
count_donate_skinVersion_anon_day.pivot_table(index=
                                              ['name','wiki','date','skinversion'], 
                                              columns=['anonymous_user']).fillna(0)

events      
anonymous_user                                 False True 
name          wiki     date       skinversion             
n-sitesupport testwiki 2020-07-16 1              0.0   2.0
                       2020-07-18 2              1.0   0.0
                       2020-07-20 1              0.0  10.0
                                  2              2.0  10.0
                       2020-07-21 2             80.0   0.0